In [2]:
from tew_code.qtft_tools import DC_QAOA, logging

import time
from scipy import sparse

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Q-Learning values. (Discounted Payoff action is in first column).
Qs = sparse.load_npz('data/Qs.npz').toarray()

# Association matrix
As = sparse.load_npz('data/As.npz').toarray()

# Provision constraints
Ls = sparse.load_npz('data/Ls.npz').toarray()

Qs_with_4_actions = Qs.copy()[:, :4]
Ls_with_4_actions = Ls.copy()[:, :4]

selected_number_of_loanees = 600

Qs = Qs_with_4_actions[:selected_number_of_loanees]
Ls = Ls_with_4_actions[:selected_number_of_loanees]
As = As[:selected_number_of_loanees, :selected_number_of_loanees]

# Number of assets
N = len(Qs)

# Number of actions
M = len(Qs[0, :])
print(N, M)

# Number of candidate solutions from QAOA
n_candidates = 100

# Number of driving cycles in QAOA. The larger this value, the better and the slower the solutions 
p = 7

# Max community size in the divide-and-conquer algorithm. 
max_community_size = 6

# Weight (epsilon) in the objective function
e = 0.2
start_time = time.time()
# Peform DC QAOA
print('-- Performing DC-QAOA --')
m = DC_QAOA(Qs, As, e, p, n_candidates, max_community_size)


600 4
-- Performing DC-QAOA --


In [3]:
m.set_communities()
group = m.groups[0]

In [4]:
Q = m.Qs[group.nodes]

In [5]:
A = m.As[group.nodes][:, group.nodes]


In [19]:
from new_basis_llp_qaoa.statevector_sim.statevector_llp import StateVectorLLP

state = "ones"
p = 1
svllp = StateVectorLLP(Q, A, 1 / 20, p, state)

In [27]:

import numpy as np
from new_basis_llp_qaoa.statevector_sim.statevector_llp import unitary_b_taylor

t_terms = svllp.hb_taylor_terms
ub = unitary_b_taylor(t_terms, np.pi).toarray()

In [32]:
4048 **2 * 2 / 1024 /1024

31.25439453125

In [7]:
res = svllp.run_optimizer(maxiter=1000)

2024-01-23 18:03:53,114 - INFO - Caching HA.
Calculating welfare term.: 100%|██████████| 6/6 [00:00<00:00, 153.19it/s]
2024-01-23 18:03:53,213 - INFO - Caching HB.
Calculating taylor series terms for HB.: 100%|██████████| 12/12 [00:02<00:00,  5.58it/s]
2024-01-23 18:03:55,382 - INFO - Getting initial state.
Optimizing:   4%|▎         | 37/1000 [01:55<50:16,  3.13s/it]  


In [8]:
res

 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -17476917859.372517
       x: [ 3.142e+00 -2.156e-04]
    nfev: 37
   maxcv: 0.0

In [20]:
from new_basis_llp_qaoa.qaoa.parameters import Parameters

#p = 1 -> 73 iterations with 1.97 seconds per iteration
#p = 3 -> more than 1000 iterations with 4.23 seconds per iteration
#p = 7 -> more than 1000 iterations with 14.78 seconds per iteration
params = res.x
betas = params[:p]
gammas = params[p:]
params = Parameters(
    gamma_list=gammas,
    beta_list=betas
)
logging.info("Evolving psi")
svllp.evolve(params)

2024-01-23 18:12:20,588 - INFO - Evolving psi
2024-01-23 18:12:20,589 - INFO - Caching HB.
Calculating taylor series terms for HB.: 100%|██████████| 12/12 [00:03<00:00,  3.55it/s]
2024-01-23 18:12:24,007 - INFO - Caching HA.
Calculating welfare term.: 100%|██████████| 6/6 [00:00<00:00, 82.54it/s]
2024-01-23 18:12:24,158 - INFO - Getting initial state.


In [22]:
svllp.psi

array([457.69486744+1929.94757749j, 457.70709744+1929.94468803j,
       457.71590167+1929.94261784j, ..., 457.74457837+1929.93587244j,
       457.75338257+1929.93380212j, 457.77706995+1929.92824272j])

 message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: 1065.1681251579857
       x: [ 1.005e+00  1.106e+00  1.038e+00  9.133e-01]
    nfev: 1000
   maxcv: 0.0
message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: -5.122875714495229e+19
       x:[0.466712 0.461325 -0.459643 0.443783 0.467750 0.468877 0.447460 0.668717 0.000326 2.464951]
    nfev: 1000
   maxcv: 0.0